# Import

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import wandb

from torch.utils.data import Subset
from torchvision.datasets import MNIST, CIFAR10, CIFAR100
from torchvision.models import resnet18
from torchvision import transforms
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from random import randint
from myModel import *
from coreFunction import *
from utilityFunction import cloneResnet

In [ ]:
download = False
dowload_path = '/Users/edoardosarri/Downloads/'

---
# Analisi

##### MNIST

In [ ]:
#import
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
train_MNIST = MNIST(root='./data', download=True, train=True, transform=transform)
test_MNIST  = MNIST(root='./data', download=True, train=False, transform=transform)

print(f'- Media MNIST: {(train_MNIST.data/255).mean()}')
print(f'- Varianza MNIST: {(train_MNIST.data/255).std()}')

In [ ]:
# stampa di informazioni
print(f'- Campi: {dir(train_MNIST)}')
print(f'- Dimensione del train set: {len(train_MNIST)}')
print(f'- Dimensione del test set: {len(test_MNIST.data)}')
print(f'- Possibili classi: {np.unique(train_MNIST.class_to_idx)}')
print(f'- Valore minimo: {train_MNIST.data.min()}')
print(f'- Valore massimo: {train_MNIST.data.max()}')
print(f'- Dimensione e tipo delle immagini: {train_MNIST.data[0].shape}')

In [ ]:
# rappresentazione immgine
image, label = train_MNIST[randint(0, len(train_MNIST))]
image = image.squeeze(0).numpy()
plt.figure(figsize=(3, 3))
plt.axis('off')
plt.title(f'Label: {label}')
plt.imshow(image, cmap='inferno')

In [ ]:
# distribuzione del train set
labels = train_MNIST.targets
palette = sns.color_palette('Spectral',10)
plt.figure()
plt.xlabel("Classe")
plt.ylabel("Numero di salmples")
sns.countplot(x=labels, hue=labels, palette=palette, legend=False)
if(download):
    percorso_file = dowload_path + 'distribuzione MNIST.pdf'
    plt.savefig(percorso_file, dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# raggruppamento trin set
n_samples = 10**4
data = train_MNIST.data[:n_samples].reshape(n_samples, -1).numpy()  # da (n,28,28) a (n,784)
labels = train_MNIST.targets[:n_samples].numpy()

tsne = TSNE(n_components=2, perplexity=30, random_state=5)
data = tsne.fit_transform(data)

plt.figure()
palette = sns.color_palette('Spectral',10)
sns.scatterplot(x=data[:, 0], y=data[:, 1], hue=labels, palette=palette, s=8)
plt.legend(title="Classi")
if(download):
    percorso_file = dowload_path + 'raggruppamento MNIST.pdf'
    plt.savefig(percorso_file, dpi=300, bbox_inches='tight')
plt.show()

##### CIFAR10

In [ ]:
#import
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.49139968, 0.48215841, 0.44653091), (0.24703223, 0.24348513, 0.26158784))
])
train_CIFAR10 = CIFAR10(root='./data', download=True, train=True, transform=transform)
test_CIFAR10 = CIFAR10(root='./data', download=True, train=False, transform=transform)

mean = (train_CIFAR10.data/255).mean(axis=(0,1,2))
std = (train_CIFAR10.data/255).std(axis=(0,1,2))
print(f'- Media CIFAR10: {mean}')
print(f'- Varianza CIFAR10: {std}')

In [ ]:
# stampa di informazioni
print(f'- Campi : {dir(train_CIFAR10)}')
print(f'- Dimensione del train set: {len(train_CIFAR10)}')
print(f'- Dimensione del test set: {len(test_CIFAR10.data)}')
print(f'- Possibili classi: {np.unique(train_CIFAR10.class_to_idx)}')
print(f'- Valore minimo: {train_CIFAR10.data.min()}')
print(f'- Valore massimo: {train_CIFAR10.data.max()}')
print(f'- Dimensione e tipo delle immagini: {train_CIFAR10[0][0].shape}')

In [ ]:
# rappresentazione immgine
images = [train_CIFAR10[randint(0, len(train_CIFAR10))] for _ in range(2)]
fig, axes = plt.subplots(2, 4, figsize=(15,6))
colors = ['Reds', 'Greens', 'Blues']
titles = ['Red', 'Green', 'Blue']
for row, (image, label) in enumerate(images):
    img = image.numpy().transpose(1,2,0)  # da (C, D, D') a (D, D', C), formato di plt
    for i in range(3):
        axes[row, i].imshow(image[i].numpy(), cmap=colors[i])
        axes[row, i].set_title(f'{titles[i]}')
        axes[row, i].axis('off')
    axes[row, 3].imshow(img * std + mean)
    axes[row, 3].set_title(f'RGB, Label: {label}')
    axes[row, 3].axis('off')
    plt.axis('off')
if(download):
    percorso_file = dowload_path + 'immagini CIFAR10.pdf'
    plt.savefig(percorso_file, dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# distribuzione del train set
labels = train_CIFAR10.targets
palette = sns.color_palette('Spectral',10)
plt.xlabel("Classe")
plt.ylabel("Numero di salmples")
sns.countplot(x=labels, hue=labels, palette=palette, legend=False)
if(download):
    percorso_file = dowload_path + 'distribuzione CIFAR10.pdf'
    plt.savefig(percorso_file, dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# raggruppamento trin set
n_samples = 10**4
data = train_CIFAR10.data[:n_samples].reshape(n_samples, -1) # da (n,128,128,3) a (n,49152)
labels = train_CIFAR10.targets[:n_samples]

pca = PCA(n_components=50, random_state=5)
data = pca.fit_transform(data)

tsne = TSNE(n_components=2, perplexity=30, method='barnes_hut', random_state=42, max_iter=1000)
data = tsne.fit_transform(data)

plt.figure()
sns.scatterplot(x=data[:, 0], y=data[:, 1], hue=labels, palette=palette, s=8)
plt.legend(title="Classi")
if(download):
    percorso_file = dowload_path + 'raggruppamento CIFAR10.pdf'
    plt.savefig(percorso_file, dpi=300, bbox_inches='tight')
plt.show()

##### CIFAR100

In [ ]:
#import
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2683, 0.2574, 0.2770))
])
train_CIFAR100 = CIFAR100(root='./data', download=True, train=True, transform=transform)
test_CIFAR100 = CIFAR100(root='./data', download=True, train=False, transform=transform)

mean = (train_CIFAR100.data / 255).mean(axis=(0, 1, 2))
std = (test_CIFAR100.data / 255).std(axis=(0, 1, 2))
print(f'- Media CIFAR100: {mean}')
print(f'- Varianza CIFAR100: {std}')

In [ ]:
# stampa di informazioni
print(f'- Campi : {dir(train_CIFAR100)}')
print(f'- Dimensione del train set: {len(train_CIFAR100)}')
print(f'- Dimensione del test set: {len(test_CIFAR100.data)}')
print(f'- Possibili classi: {np.unique(train_CIFAR100.class_to_idx)}')
print(f'- Valore minimo: {train_CIFAR100.data.min()}')
print(f'- Valore massimo: {train_CIFAR100.data.max()}')
print(f'- Dimensione e tipo delle immagini: {train_CIFAR100[0][0].shape}')

In [ ]:
# rappresentazione immgine
images = [train_CIFAR100[randint(0, len(train_CIFAR100))] for _ in range(2)]
fig, axes = plt.subplots(2, 4, figsize=(15,6))
colors = ['Reds', 'Greens', 'Blues']
titles = ['Red', 'Green', 'Blue']
for row, (image, label) in enumerate(images):
    img = image.numpy().transpose(1,2,0)  # da (C, D, D') a (D, D', C), formato di plt
    for i in range(3):
        axes[row, i].imshow(image[i].numpy(), cmap=colors[i])
        axes[row, i].set_title(f'{titles[i]}')
        axes[row, i].axis('off')
    axes[row, 3].imshow(img * std + mean)
    axes[row, 3].set_title(f'RGB, Label: {label}')
    axes[row, 3].axis('off')
    plt.axis('off')
if(download):
    percorso_file = dowload_path + 'immagini CIFAR10.pdf'
    plt.savefig(percorso_file, dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# distribuzione del train set
labels = train_CIFAR100.targets
palette = sns.color_palette('Spectral',100)
plt.xlabel("Classe")
plt.ylabel("Numero di salmples")
sns.countplot(x=labels, hue=labels, palette=palette, legend=False).set_xticks([])
if(download):
    percorso_file = dowload_path + 'distribuzione CIFAR10.pdf'
    plt.savefig(percorso_file, dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# raggruppamento trin set
n_samples = 10**4
data = train_CIFAR100.data[:n_samples].reshape(n_samples, -1)  # da (n,128,128,3) a (n,49152)
labels = train_CIFAR100.targets[:n_samples]

pca = PCA(n_components=50, random_state=5)
data = pca.fit_transform(data)

tsne = TSNE(n_components=2, perplexity=30, method='barnes_hut', random_state=42, max_iter=1000)
data = tsne.fit_transform(data)

plt.figure()
palette = sns.color_palette('Spectral',100)
sns.scatterplot(x=data[:, 0], y=data[:, 1], hue=labels, palette=palette, s=8, legend=False)
if(download):
    percorso_file = dowload_path + 'raggruppamento CIFAR100.pdf'
    plt.savefig(percorso_file, dpi=300, bbox_inches='tight')
plt.show()

---
# MLP

##### w/o residual connection

In [ ]:
layers = [784, 1024, 256, 64, 10]
model = MLP(layers)
pipeline(model, train_MNIST, test_MNIST, 'MNIST', 'MLP_4', lr=1e-4, evaluating_grad=True)

layers = [784] + [1024] + [256] + [64]*10 + [10]
model = MLP(layers)
pipeline(model, train_MNIST, test_MNIST, 'MNIST', 'MLP_13', lr=1e-4, evaluating_grad=True)

layers = [784] + [1024] + [256] + [64]*20 + [10]
model = MLP(layers)
pipeline(model, train_MNIST, test_MNIST, 'MNIST', 'MLP_23', lr=1e-4, evaluating_grad=True)

layers = [784] + [1024] + [256] + [64]*30 + [10]
model = MLP(layers)
pipeline(model, train_MNIST, test_MNIST, 'MNIST', 'MLP_33', lr=1e-4, evaluating_grad=True)

##### w/ residual connection

In [ ]:
layers = [784, 1024, 256, 64, 10]
model = MLP(layers, res=True)
pipeline(model, train_MNIST, test_MNIST, 'MNIST', 'MLP_res_5', lr=1e-4, evaluating_grad=True)

layers = [784] + [1024] + [256] + [64]*10 + [10]
model = MLP(layers, res=True)
pipeline(model, train_MNIST, test_MNIST, 'MNIST', 'MLP_res_15', lr=1e-4, evaluating_grad=True)

layers = [784] + [1024] + [256] + [64]*30 + [10]
model = MLP(layers, res=True)
pipeline(model, train_MNIST, test_MNIST, 'MNIST', 'MLP_res_35', lr=1e-4, evaluating_grad=True)

layers = [784] + [1024]*15 + [256]*20 + [64]*15 + [10]
model = MLP(layers, res=True)
pipeline(model, train_MNIST, test_MNIST, 'MNIST', 'MLP_res_55', lr=1e-4, evaluating_grad=True)

layers = [784] + [2048]*30 + [1024] + [256]*20 + [128]*10 + [64]*30 + [10]
model = MLP(layers, res=True)
pipeline(model, train_MNIST, test_MNIST, 'MNIST', 'MLP_res_96', lr=1e-4, evaluating_grad=True)


---
# CNN

##### w/o residual connection

In [ ]:
pipeline(CNN1(), train_CIFAR10, test_CIFAR10, 'CIFAR10', 'CNN_10', evaluating_grad=True)
pipeline(CNN2(), train_CIFAR10, test_CIFAR10, 'CIFAR10', 'CNN_13', evaluating_grad=True)
pipeline(CNN3(), train_CIFAR10, test_CIFAR10, 'CIFAR10', 'CNN_30', evaluating_grad=True)

##### w/ residual connection

In [ ]:
pipeline(CNN2_res(), train_CIFAR10, test_CIFAR10, 'CIFAR10', 'CNN_res_16', evaluating_grad=True)
pipeline(CNN3_res(), train_CIFAR10, test_CIFAR10, 'CIFAR10', 'CNN_res_35', evaluating_grad=True)
pipeline(CNN_deep_res(), train_CIFAR10, test_CIFAR10, 'CIFAR10', 'CNN_res_56', evaluating_grad=True)

---
# Fine tune and optimizing

##### Baseline

In [ ]:
val_size = 5000
I = np.random.permutation(len(train_CIFAR100))
val_CIFAR100 = Subset(train_CIFAR100, I[:val_size])
train_CIFAR100 = Subset(train_CIFAR100, I[val_size:])

print(f'- Dimensione del validation set: {len(val_CIFAR100)}')
print(f'- Dimensione del train set: {len(train_CIFAR100)}')

In [ ]:
resnet_c10 = resnet18(weights=None)
resnet_c10.fc = nn.Linear(512, 10)
pipeline(resnet_c10, train_CIFAR10, test_CIFAR10, 'CIFAR100', 'train_resnet')
extract_and_classical(resnet_c10, train_CIFAR100, test_CIFAR100)

##### Fine tuning

In [ ]:
# learning rate troppo alto

resnet = cloneResnet(resnet_c10)

for param in resnet.parameters():
    param.requires_grad = False
for param in resnet.fc.parameters():
    param.requires_grad = True
pipeline(resnet, train_CIFAR100, val_CIFAR100, 'CIFAR100', 'fine_tune_1', classi=100)

for param in resnet.layer4.parameters():
    param.requires_grad = True
pipeline(resnet, train_CIFAR100, val_CIFAR100, 'CIFAR100', 'fine_tune_2', classi=100)

In [ ]:
# learning rate più basso

resnet = cloneResnet(resnet_c10)

for param in resnet.parameters():
    param.requires_grad = False
for param in resnet.fc.parameters():
    param.requires_grad = True
pipeline(resnet, train_CIFAR100, val_CIFAR100, 'CIFAR100', 'fine_tune_1_lr', classi=100)

for param in resnet.layer4.parameters():
    param.requires_grad = True
pipeline(resnet, train_CIFAR100, val_CIFAR100, 'CIFAR100', 'fine_tune_2_lr', lr=1e-5, classi=100)

for param in resnet.layer3.parameters():
    param.requires_grad = True
pipeline(resnet, train_CIFAR100, val_CIFAR100, 'CIFAR100', 'fine_tune_3_lr', lr=1e-5, classi=100)

##### Early stopping

In [ ]:
resnet = cloneResnet(resnet_c10)
pipeline(resnet, train_CIFAR100, val_CIFAR100, 'CIFAR100', 'early stopping', early_stopping=True, lr=5e-3, classi=100)

##### Optimizer

In [ ]:
resnet = cloneResnet(resnet_c10)
opt = torch.optim.Adam(resnet.parameters(), lr=5e-4)
pipeline(resnet, train_CIFAR100, val_CIFAR100, 'CIFAR100', 'adam', early_stopping=True, classi=100, opt=opt)

resnet = cloneResnet(resnet_c10)
opt = torch.optim.SGD(resnet.parameters(), lr=5e-4)
pipeline(resnet, train_CIFAR100, val_CIFAR100, 'CIFAR100', 'sgd', early_stopping=True, classi=100, opt=opt)

resnet = cloneResnet(resnet_c10)
opt = torch.optim.AdamW(resnet.parameters(), lr=5e-4)
pipeline(resnet, train_CIFAR100, val_CIFAR100, 'CIFAR100', 'adamW', early_stopping=True, classi=100, opt=opt)

resnet = cloneResnet(resnet_c10)
opt = torch.optim.RMSprop(resnet.parameters(), lr=5e-4)
pipeline(resnet, train_CIFAR100, val_CIFAR100, 'CIFAR100', 'RMSprop', early_stopping=True, classi=100, opt=opt)


resnet = cloneResnet(resnet_c10)
opt = torch.optim.Adam(resnet.parameters(), lr=1e-5)
pipeline(resnet, train_CIFAR100, val_CIFAR100, 'CIFAR100', 'adam2', early_stopping=True, classi=100, opt=opt, epochs=50)

resnet = cloneResnet(resnet_c10)
opt = torch.optim.SGD(resnet.parameters(), lr=1e-5)
pipeline(resnet, train_CIFAR100, val_CIFAR100, 'CIFAR100', 'sgd2', early_stopping=True, classi=100, opt=opt, epochs=50)

resnet = cloneResnet(resnet_c10)
opt = torch.optim.AdamW(resnet.parameters(), lr=1e-5)
pipeline(resnet, train_CIFAR100, val_CIFAR100, 'CIFAR100', 'adamW2', early_stopping=True, classi=100, opt=opt, epochs=50)

resnet = cloneResnet(resnet_c10)
opt = torch.optim.RMSprop(resnet.parameters(), lr=1e-5)
pipeline(resnet, train_CIFAR100, val_CIFAR100, 'CIFAR100', 'RMSprop2', early_stopping=True, classi=100, opt=opt, epochs=50)

##### Iperparametri

In [ ]:
def mysweep():
    with wandb.init() as run:
        config = wandb.config
        run_name = f'resnset(lr: {round(config.lr,7)}, bs: {config.batch_size})'
        resnet = cloneResnet(resnet_c10)
        pipeline(resnet, train_CIFAR100, val_CIFAR100, 'CIFAR100', run_name, classi=100, lr=config.lr, batch_size=config.batch_size, early_stopping=True)

sweep_config = {
    'method': 'bayes',
    'metric': {'name': 'test_acc', 'goal': 'maximize'},
    'parameters': {
        'lr': {'distribution': 'log_uniform_values', 'min': 1e-6, 'max': 1e-2},
        'batch_size': {'values': [16, 32, 64, 128, 256]}}
}
    
sweep_id = wandb.sweep(sweep = sweep_config, project='CIFAR100')
wandb.agent(sweep_id, function=mysweep, count=12)

##### Dropout and generalization

In [ ]:
resnet = cloneResnet(resnet_c10)
resnet.fc = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(512, 100)
)

pipeline(resnet, train_CIFAR100, val_CIFAR100, 'CIFAR100', 'dropout', epochs=100, early_stopping=True, classi=100, batch_size=64, lr=0.009306381822230696)

dl_test = get_dataloader(test_CIFAR100, 64)
acc_test = testing_acc(resnet, dl_test, 100)
print(f'Accuracy sul test set: {acc_test}')